# YAHOO電影爬蟲練習
## 這個作業主要是練習爬取YAHOO電影的資訊，主要會用到基本的GET method，以及解析伺服器回傳的html，從中擷取我們想要的資料。

### 這個範例示範如何爬取上映中的電影資訊。先訪問YAHOO電影首頁：https://movies.yahoo.com.tw ，點選「上映中」的分頁，先確認上映中的電影總共有幾筆，然後逐頁爬取。

In [16]:
import requests
import re
from bs4 import BeautifulSoup

In [17]:
# 先觀察一下目前上映中的電影數量
url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'html.parser')
html = soup.find("div", attrs={'class':'release_box'})  # 尋找正在上映中的全部電影筆數，其所在的tag
print("正在上映中總共: ", html.p.string)

正在上映中總共:  共75筆，目前顯示1~10筆


### 爬取所有上映中的電影資料：電影名稱、期待度、滿意度、簡介等等。

In [18]:
# 接下來要爬取每一頁的電影介紹
# 先點選網頁最底部的下一頁，觀察其網址變化。然後我們把頁數當作變數傳入，用迴圈爬取。
max_page = 2
for page_number in range(1, max_page+1, 1):
    url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
    payload = {'page':str(page_number)}
    resp = requests.get(url, params=payload)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'lxml')

    # 把電影介紹擷取出來
    movie_list = soup.find("ul", attrs={"class":"release_list"}).find_all("li")
    for p in movie_list:
        print("----------------------------------------------------------------------")
        # 電影名稱
        movie_name = p.find("div", attrs={"class":"release_movie_name"})
        print("電影名稱：", movie_name.a.string)
        
        # 定位電影評價資訊欄位
        level_box = movie_name.find("dl", attrs={"class":"levelbox"})
        
        # 期待度
        expectation = level_box.find("div", attrs={"class":"leveltext"})
        print("期待度：", expectation.span.string)
        
        # 滿意度
        satisfaction = level_box.find("span", attrs={"class":"count"})
        print("滿意度：", satisfaction["data-num"])  # 滿意度可以從"data-num"這個屬性擷取
        
        # 簡介
        movie_info = p.find("div", attrs={"class":"release_text"})
        print(movie_info.span.string)


----------------------------------------------------------------------
電影名稱： 
                  音速小子
期待度： 69%
滿意度： 4.1

                  ★《死侍》、《玩命關頭》幕後團隊最新力作
 ★智慧大反派金凱瑞端出高科技武器 PK 全新超級快英雄音速小子超能量
 
派拉蒙影業2020年2月推出一部真人動作冒險喜劇《音速小子》，根據全球知名SEGA電玩改編，由《玩命關頭》和《死侍》製作團隊特別量身打造經典角色，一隻家喻戶曉的亮藍色刺蝟，全新超級快英雄“音速小子”，準備飆速新世界!!《音速小子》敘述誤闖地球的音速小子和新認識的人類朋友湯姆華卓斯基（詹姆斯馬斯登 飾），兩人要聯手阻止邪惡的蛋頭博士（搞笑天王金凱瑞 飾），利用音速小子的巨大力量來統治全世界。這部電影導演是傑夫富勒，其他演員還有蒂卡桑普特和為音速小子配音的班許瓦茲。
                
----------------------------------------------------------------------
電影名稱： 
                  李察朱威爾事件
期待度： 92%
滿意度： 4.3

                  由克林伊斯威特執導，根據真實事件改編，《李察朱威爾事件》一片描述當不實的報導被當作事實時，真相會如何被遮掩的故事。  
 
「奧林匹克百年公園有一枚炸彈，你們只有三十分鐘。」世人第一次知道李察朱威爾這號人物是在1996年的亞特蘭大爆炸案中，身為第一個發現炸彈並報警的保全，李察朱威爾及時報警並協助疏散群眾，拯救無數人命的英勇行為使他成為一名英雄，但是幾天後，這個想要當一名執法人員的保全卻成為美國聯邦調查局的頭號嫌犯，受到媒體和大眾的詆毀與唾棄，他的人生也瞬間分崩離析。深知自己是無辜的李察朱威爾向反體制的獨立律師華森布萊恩求助，希望他能夠證明自己的清白，然而華森布萊恩在奮力為李察朱威爾洗清罪名的過程中卻發現，他一個人的力量是如此薄弱，因為這次除了必須對抗聯邦調查局、喬治亞州調查局及亞特蘭大市警署的強大勢力，與此同時，他也必須說服李察不能單純地信任這些試圖毀了他人生的人們。 
                
---